<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week9.do.txt --no_mako -->
<!-- dom:TITLE: March 20-24, 2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories -->

# March 20-24, 2023: Quantum Computing, Quantum Machine Learning and Quantum Information Theories
**Morten Hjorth-Jensen**, Department of Physics, University of Oslo and Department of Physics and Astronomy and Facility for Rare Isotope Beams, Michigan State University

Date: **March 20-24**

1. Preparing the Lipkin model for  VQE quantum computing

2. [Discussion of the project](https://github.com/CompPhysics/QuantumComputingMachineLearning/tree/gh-pages/doc/Projects/2023/Project1)

3. [Video of lecture](https://youtu.be/ZKv8pEy3Y_4)

## Reminder on the Lipkin model and Variational Quantum Eigensolver (VQE)

See lecture notes (jupyter-notebook)
from the week of March 13-17 at <https://github.com/CompPhysics/QuantumComputingMachineLearning/blob/gh-pages/doc/pub/week8/ipynb/week8.ipynb>.

## Quantum computing and solving  the eigenvalue problem for the Lipkin model

We turn now to a simpler variant of the Lipkin model without the $W$-term and a total spin of $J=1$ only as maximum value of the spin.
This corresponds to a system with $N=2$ particles (fermions in our case).
Our Hamiltonian is given by the quasispin operators (see below)

$$
\hat{H} = \epsilon\hat{J}_z -\frac{1}{2}V(\hat{J}_+\hat{J}_++\hat{J}_-\hat{J}_-).
$$

[As discussed
previously](https://github.com/CompPhysics/QuantumComputingMachineLearning/blob/gh-pages/doc/pub/week8/ipynb/week8.ipynb),
the quasispin operators act like lowering and raising angular momentum
operators.

With these properties we can calculate the Hamiltonian
matrix for the Lipkin model by computing the various matrix elements

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
\langle JJ_z|H|JJ_z'\rangle,
\label{_auto1} \tag{1}
\end{equation}
$$

where the non-zero elements are given by

$$
\begin{split}
\langle JJ_z|H|JJ_z'\rangle & = \epsilon J_z\\
\langle JJ_z|H|JJ_z'\pm 2\rangle & = \langle JJ_z\pm 2|H|JJ_z'\rangle \\ &= -\frac{1}{2}VC,
\end{split}
$$

where $C$ is the Clebsch-Gordan coefficients (from the raising and lowering operators) one gets when
$J_{\pm}^2$ operates on the state $|JJ_z\rangle$.  Using the above
definitions we can calculate the exact solution to the Lipkin model.

With the $V$-interaction terms, we obtain the following Hamiltonian matrix

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}
\begin{pmatrix}-\epsilon & 0 & -V\\
 0&0&0\\
 -V&0&\epsilon
\end{pmatrix}
\label{_auto2} \tag{2}
\end{equation}
$$

The following **python** code sets up the above matrix and finds the pertinent eigenvalues.

In [1]:
%matplotlib inline

import numpy as np
import qiskit
from qiskit.visualization import circuit_drawer
from qiskit.quantum_info import Statevector
from matplotlib.pyplot import figure
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer, assemble
from qiskit.providers.aer.noise import NoiseModel
import pylatexenc
from qiskit.algorithms import VQE
from qiskit.utils import QuantumInstance
from qiskit.opflow import X, Z, I, Y
from qiskit.circuit import Parameter
from qiskit.algorithms.optimizers import ADAM
from qiskit.opflow import AerPauliExpectation
from qiskit import IBMQ
import cmath
import pandas as pd
from scipy.sparse import diags
import numpy.linalg as LA
import matplotlib.pyplot as plt
from IPython.display import Image
import warnings
warnings.filterwarnings('ignore')
pi=np.pi

In [2]:
#function that sorts eigenvalues with its eigenvectors in accending order
def eigen(A):
    eigenValues, eigenVectors = LA.eig(A)
    idx = np.argsort(eigenValues)
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    return (eigenValues, eigenVectors) 

#one body expectation value
def one_body(E,N):
  k = N/2
  m = np.arange(-k,k+1,1) # Since the collective space is Omega+1
  return E*np.diag(m) #return a matrix where its diagonal elemens are epsilon*K_0

#two body expectation value
def two_body(V,N):
  k = N/2
  m = np.arange(-k,k+1,1)
  left =np.zeros(len(m)-2,dtype=complex)
  right = np.zeros(len(m)-2,dtype=complex)
  diag = np.zeros(len(m),dtype=complex)
  for i in range(len(left)):
    CG = cmath.sqrt(k*(k+1)-(m[i]+2)*(m[i]+1))*cmath.sqrt(k*(k+1)-m[i]*(m[i]+1)) #calculate Clebsch-Gordan Coefficients 
    left[i] = CG
    right[i] = CG
  k = [left,diag,right]
  offset = [-2,0,2]
  return -0.5*V*diags(k,offset).toarray() #return a matrix where its off digonal elements are (1/2)V(K^2_+ + K^2_-)

#full expectation value
def quasi_spin(E,V,N):
  ob = one_body(E,N)
  tb = two_body(V,N)
  H = ob+tb 
  e,v = eigen(H) # find the eigenvalues of the Hamiltonian
  return e,H

#converts chi to V
def Vp(E,omega,chi):
  return (chi*E)/(omega-1)

In [3]:
#parameters
E = 1
chi = np.arange(0,2.1,0.1)
omega = 2

EV0 = []
EV1 = []
EV2 = []
Ham = []
for i in chi:
  v = Vp(E,omega,i)
  EigenV,H = quasi_spin(E,v,omega) #return eigenvalues and Hamiltonian
  Ham.append(H)
  EV0.append(EigenV[0])
  EV1.append(EigenV[1])
  EV2.append(EigenV[2])

In [4]:
matrix = pd.DataFrame(Ham[5].real)
print('Hamiltonian matrix')
matrix.head()

In [5]:
plt.plot(chi,EV0)
plt.plot(chi,EV1)
plt.plot(chi,EV2)
plt.xlabel('$\chi$')
plt.ylabel('Energy')
plt.title('$\Omega=2$ exact Lipkin Model')

### Quantum Circuit, rewriting the Lipkin model in terms of Pauli matrices

To solve the Lipkin model on a quantum computer we have to solve
Schrodinger's equation. To achieve this, we will use the Variational
Quantum Eigensolver (VQE) [discussed
earlier](https://github.com/CompPhysics/QuantumComputingMachineLearning/blob/gh-pages/doc/pub/week8/ipynb/week8.ipynb).

Before we proceed however, we need to rewrite the quasispin operators in terms of Pauli spin matrices/operators.

We take the liberty here of reminding you of some of the derivations done previously.
We defined the number operator as

$$
N=\sum_{n\sigma}a^\dagger_{n\sigma}a_{n\sigma},
$$

which commutes with the Lipkin Hamiltonian. This can be seen by
examining the Lipkin model Hamiltonian and noticing that the one-body
part simply counts particles while the two-body term moves particles
in pairs. Thus, the Hamiltonian conserves particle number. To find
more symmetries we rewrote the Lipkin Hamiltonian in terms of $SU(2)$ quasispin
operators

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation}
H = \epsilon J_z + \frac{1}{2}V(J^2_++J^2_-),
\label{_auto3} \tag{3}
\end{equation}
$$

via the mappings

$$
J_z=\sum_{n}j_z^{(n)},
$$

and

$$
J_\pm=\sum_nj^{(n)}_{\pm},
$$

where we have the onebody operators

$$
j_z^{(n)}=\frac{1}{2}\sum_{\sigma}\sigma a^\dagger_{n\sigma}a_{n\sigma},
$$

and

$$
j^{(n)}_{\pm}=a^\dagger_{n\pm}a_{n\mp}.
$$

These operators obey the $SU(2)$ commutation relations

$$
[J_+,J_-]=2J_z,
$$

and

$$
[J_z,J_\pm]=\pm J_\pm.
$$

Here the ladder operators are defined as $J_{\pm}= J_x\pm iJ_y$. With this rewriting, we can see that the total spin operator $J^2$, which is defined as

$$
J^2= J^2_x+J^2_y+J^2_z =
\frac{1}{2}\{J_+,J_-\}+J_z^2,
$$

commutes with the Hamiltonian since the Hamiltonian.
We note also that the rotation operator

$$
R=e^{i\phi J_z},
$$

commutes with the Hamiltonian, which can be explained as follows. Writing $J_z$ as

$$
J_z=\frac{1}{2}(N_+-N_-),
$$

where $N_\pm=\sum_{n\pm}a^\dagger_{n\pm}a_{n\pm}$, allows us to see that it measures half the difference between the number of particles in the upper and lower levels. Thus, the possible eigenvalues $r$ of the signature operator are

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation}
r=+1,  j_z=2n 
\label{_auto4} \tag{4}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto5"></div>

$$
\begin{equation} 
r=+i,  j_z=2n+\frac{1}{2} 
\label{_auto5} \tag{5}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto6"></div>

$$
\begin{equation} 
r=-1,  j_z=2n+1 
\label{_auto6} \tag{6}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto7"></div>

$$
\begin{equation} 
r=-i,  j_z=2n+\frac{3}{2} 
\label{_auto7} \tag{7}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto8"></div>

$$
\begin{equation} 
\label{_auto8} \tag{8}
\end{equation}
$$

for $n\in\mathbb{Z}$. Note that $r$ is real or imaginary if the number
of particles $N$ is even or odd, respectively. Since, as discussed
above, the Lipkin Hamiltonian conserves $N$, $r$ cannot jump between
being real and imaginary. Additionally, because particles must be
moved in pairs, and $J_z$ measures half the difference between
particles in the upper and lower levels, $j_z$ can only change by as

$$
j_z\rightarrow \frac{1}{2}[(N_+\pm 2n)-(N_-\mp 2n)]
$$

or $j_z\rightarrow J_z\pm2n$.

To solve the Lipkin model with a quantum computer, the first step is
to map the system to a set of qubits. We will restrict ourselves here
to the half-filled case where the number of particles $N$ equals the
degeneracy of the states $\Omega$. One could assign each possible
state $(n,\sigma)$ a qubit such that the qubit being in the state
$\vert 1\rangle$ or $\vert 0\rangle$ would imply that the state
$(n,\sigma)$ is occupied or unoccupied, respectively. This mapping
scheme (which we will call occupation mapping) requires 2$\Omega$
qubits.

However, because there are only two energy levels in the Lipkin model,
any other natural mapping is possible. In this mapping scheme (which
we will call level mapping) each doublet ($(n,+1)$, $(n,-1)$) would be
assigned a qubit such that the qubit being in the state $\vert 0\rangle$ or
$\vert 1\rangle$ would imply that the particle is in the $(n,+1)$ or $(n,-1)$
state, respectively. Note that these are the only two possible
configurations of the doublet as we are restricting ourselves to the
half-filled case and the Lipkin Hamiltonian only moves particles
between energy levels, not degenerate states. Thus the level mapping
only requires $\Omega$ qubits which is half that of the occupation
mapping. Additionally, any ansatz that would restrict the minimization
search to the correct subspace of constant Hamming weight $N$ requires
at most, only two qubit gates. This is because moving a pair of
particles in this scheme only changes the state of two doublets (and
therefore qubits). That is, it only takes a two-qubit gate to change
between the states $\vert 00\rangle$ and $\vert 11\rangle$, for example. As an
efficient decomposition two-qubit gates is known, the ansatz for this
mapping would be shorter (and thus less noisy) than that of the
previous mapping.

One could imagine a third mapping scheme which would require even less
qubits in which each of the possible states in the spin basis
$\vert JJ_z\rangle$ is mapped to a single qubit. In this spin mapping, there
are only $2J+1$ possible states (since $J_z=-J,-J+1,...,J-1,J$) for
each value of $J$. And, since the Hamiltonian is block diagonal (with
a different block for each $J$) the eigenvalues of the Hamiltonian are
simply the eigenvalues of each block, which may be calculated
separately. Since the maximum value of $J$ is $J_{\text{max}}=N/2$,
the largest number of qubits would be
$2J_{\text{max}}+1=N+1$. However, $\left\lfloor N/2 \right\rfloor$
different circuit would need to be used for minimization for all
possible values of $J$, to explore the entire Hilbert space. (The
minimum of the set of minimum energies that each circuit finds would
be the ground state energy of the entire system.) This increases,
linearly, the amount of time required to find the ground state energy.

After reviewing the three possible mappings, it is our view that the
level mapping is the best suited for NISQ era
devices given its low qubit count and ability to search the entire
relevant Hilbert space with one circuit (which reduces time to
solution) and the fact that at most, only two-qubit gates are required
of the ansatz, leading to shorter depth (and thus less noisy)
circuits.

The Hamiltonian takes the form

<!-- Equation labels as ordinary links -->
<div id="_auto9"></div>

$$
\begin{equation}
H=\epsilon J_z + \frac{1}{2}V(J^2_++J^2_-).
\label{_auto9} \tag{9}
\end{equation}
$$

Plugging the mapping from the total $J$ operators to the individual one-body $j$ operators yields

<!-- Equation labels as ordinary links -->
<div id="_auto10"></div>

$$
\begin{equation}
H = \epsilon\sum_{n}j_z^{(n)} + \frac{1}{2}V\left[\left(\sum_nj^{(n)}_{+}\right)^2+\left(\sum_nj^{(n)}_{-}\right)^2\right]
\label{_auto10} \tag{10}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto11"></div>

$$
\begin{equation} 
= \epsilon\sum_{n}j_z^{(n)} + \frac{1}{2}V\sum_{n,m}\left(j^{(n)}_+j^{(m)}_++j^{(n)}_-j^{(m)}_-\right)
\label{_auto11} \tag{11}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto12"></div>

$$
\begin{equation} 
= \epsilon\sum_{n}j_z^{(n)} + 2V\sum_{n<m}\left(j^{(n)}_xj^{(m)}_x-j^{(n)}_yj^{(m)}_y\right),
\label{_auto12} \tag{12}
\end{equation}
$$

where we have used the definitions

$$
j_{\pm}^{(n)}=j_x^{(n)}\pm ij_y^{(n)}.
$$

To convert to Pauli matrices, we make the transformations

$$
j_x^{(n)} \rightarrow X_n/2,
$$

and

$$
j_y^{(n)} \rightarrow Y_n/2,
$$

and finally

$$
j_z^{(n)} \rightarrow Z_n/2,
$$

which preserve the above $SU(2)$  commutation relations.
The factor of $1/2$
is due to the eigenvalues of the Pauli matrices being $\pm 1$
while we are dealing with spin $1/2$ particles.

This transforms our Hamiltonian into

$$
H=\frac{1}{2}\epsilon\sum_{k=1}^nZ_k+\frac{1}{2}V\sum_{n\neq j=1}^N(X_kX_j-Y_kY_j).
$$

With this form, we can clearly see that the first (one-body) term in
the Hamiltonian returns the energy $-\epsilon/2$ or $+\epsilon/2$ if
the qubit representing the particle of a doublet is in the ground
($\vert 1\rangle$) or excited ($\vert 0\rangle$) state,
respectively. The action of the second (two-body) term in the
Hamiltonian can be determined by noting that

<!-- Equation labels as ordinary links -->
<div id="_auto13"></div>

$$
\begin{equation}
\frac{1}{2}(XX-YY)\vert 00\rangle = \vert 11\rangle,
\label{_auto13} \tag{13}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto14"></div>

$$
\begin{equation} 
\frac{1}{2}(XX-YY)\vert 01\rangle = 0,
\label{_auto14} \tag{14}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto15"></div>

$$
\begin{equation} 
\frac{1}{2}(XX-YY)\vert 10\rangle = 0,
\label{_auto15} \tag{15}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto16"></div>

$$
\begin{equation} 
\frac{1}{2}(XX-YY)\vert 11\rangle = \vert 00\rangle.
\label{_auto16} \tag{16}
\end{equation}
$$

That is, the two-body term moves a pair of particles between the
ground states $\vert 00\rangle$ and the excited states $\vert
11\rangle$ of their respective doublets.

To construct an efficient ansatz, we must determine the subspace
within which the Hamiltonian lives. To begin, note that particles are
only ever moved between energy levels in pairs.
Further,
note that the Hamiltonian's coefficients ($\epsilon$ and $V$) are
state independent (do not depend on the indices $n$ or $m$) as the
states labeled by these indices are degenerate and thus have the same
energy level. Thus, the Hamiltonian treats all states with the same
number of excited particles (Hamming weight of the state) as the
same. Therefore, the following ansatz forms exactly cover the subspace
within which the $N$-degenerate Hamiltonian explores:

<!-- Equation labels as ordinary links -->
<div id="_auto17"></div>

$$
\begin{equation}
\vert \psi_{\text{even}}\rangle=\sum_{k=0}^{\lfloor n/2 \rfloor}c_{2k}\vert D^n_{2k}\rangle,
\label{_auto17} \tag{17}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto18"></div>

$$
\begin{equation} 
\vert \psi_{\text{odd}}\rangle=\sum_{k=0}^{\lfloor n/2 \rfloor}c_{2k+1}\vert D^n_{2k+1}\rangle.
\label{_auto18} \tag{18}
\end{equation}
$$

Here $\vert D^n_k\rangle$ represents a Dicke state which is defined as equal superposition of all $n$-qubit states with Hamming weight $k$. That is

<!-- Equation labels as ordinary links -->
<div id="_auto19"></div>

$$
\begin{equation}
\vert D^n_k\rangle= \frac{1}{\sqrt{{n \choose k}}}\sum_{x\in h^n_k}\vert x\rangle,
\label{_auto19} \tag{19}
\end{equation}
$$

where $h^n_k= \{\vert x\rangle \ | \ \text{l}(x) = n, \ \text{wt}(x) = k\}$. There are two ways we can think of to prepare such ansatz: The first is to prepare them exactly as it is known how to deterministically prepare Dicke states with linear depth. The reference provides an algorithm for preparing a set of gates $U^n_k$ that prepares a Dicke state from a product state of Hamming weight $k$; that is

<!-- Equation labels as ordinary links -->
<div id="_auto20"></div>

$$
\begin{equation}
U^n_k\vert 1\rangle^{\otimes k}\vert 0\rangle^{\otimes n-k}=\vert D^n_k\rangle.
\label{_auto20} \tag{20}
\end{equation}
$$

It then describes how to one can create an arbitrary superposition of Dicke states, which we modify here to restrict ourselves to a Hamming weight of constant parity. The circuit to construct such a state (for the $k=6$ case, as an example) is discussed next week.

**Plans for the week March 27-31.**

1. Discussion of the project and how to implement the VQE for the simpler matrix problems

2. Solving the Lipkin model with VQE